### Data: 

* 453? xml files with songs
* all these files come from the 21WDW (scouts group) songbook 
* each song has a title and lyrics, but also (not all of them) artists/ creator and other information
* I am interested in the lyrics of the songs and I try split into clusters

In [3]:
#all imports, remove unused on the finish

# data manipulation and visualization
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.figure_factory as ff
import seaborn as sns
import os

# genaral libraries
import requests
import re
from math import sin, cos, sqrt, atan2, radians

# algorithms
from sklearn.datasets import make_blobs, make_moons, make_circles
from sklearn.cluster import KMeans, AgglomerativeClustering, DBSCAN, MiniBatchKMeans
from sklearn.neighbors import NearestNeighbors

# performance metrics
from sklearn.metrics import silhouette_score

# for text representation & data preparation
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.preprocessing import Normalizer
from sklearn.metrics.pairwise import cosine_similarity

# agglomerative clustering
from scipy.spatial.distance import pdist, squareform
from scipy.cluster.hierarchy import linkage
from scipy.cluster.hierarchy import dendrogram
import scipy.cluster.hierarchy as sch
import scipy.spatial as ss


#to display the entire width of the data frame
pd.set_option('display.max_colwidth', None)

#to display the diagram
from sklearn import set_config
set_config(display="diagram")


In [11]:
songs=os.listdir("Data/songs/")
songs=[x for x in songs if os.path.splitext(x)[1]==".xml"
songs #list of xml files in Data directory

['Szesc_blota_stop.xml',
 'Balonik.xml',
 'niepatrznato.xml',
 'Madonna.xml',
 'Nierozdziobai.xml',
 'Przechyly.xml',
 'Znow_wedrujemy.xml',
 'Makatka_z_aniolem.xml',
 'Bieszczadzki_trakt.xml',
 'Ach_kiedy_znowu_rusza.xml',
 'Wiatr,_wiosenny_gitarzysta.xml',
 'Dokad_nas_zaprowadzisz_Panie.xml',
 'Jedyne_co_mam.xml',
 'Miedzy_nami_tyle_sniegu.xml',
 'Dylan_--_pastisz.xml',
 'wasza_wysokosc_kobieto.xml',
 'Chlopaki_nie_placza.xml',
 'Gonic_marzenia.xml',
 'Przyjaciolko.xml',
 'Tawerna_pod_pijana_zgraja.xml',
 'Pocaluj_noc.xml',
 'Zwiewnosc.xml',
 'Zegluj.xml',
 'Easy_Rider.xml',
 'Wielki_Woz.xml',
 'Pan_Kmicic.xml',
 'Broadway.xml',
 'Juz_rozpalilo_sie_ognisko.xml',
 'ciwszyscyludzie.xml',
 'Jesienne_wspominanki.xml',
 'Jaka_cene.xml',
 'Lekcja_historii_klasycznej.xml',
 'Miasto.xml',
 'Placz_moja_wodo.xml',
 'Szkic_do_portretu.xml',
 'Piosenka_dla_Wojtka_Bellona.xml',
 'Wojtka_Bellona_ostatnia_ziemska_podroz_do_Wlodawy.xml',
 'Zabierz_mnie_na_stacje.xml',
 'Piosenka_dla_juniora_i_jego_g

In [12]:
len(songs)

453

In [13]:
# File parsing song in xml (I created this code to generate a epub songbook)

from lxml import etree
from enum import Enum
from distutils.util import strtobool

class RowChunk:  # supports chords
    def __init__(self, chord='', content=None):
        self.chord = chord
        if content is None:
            self.content = ''
        else:
            self.content = content.strip('\t\n')


class RowType(Enum):
    FIRST = 'F'
    FIRST_SPECIAL = 'FS'
    MIDDLE = 'M'
    LAST = 'L'
    SINGLE = 'S'


class Row:
    def __init__(self, row_type=RowType.MIDDLE, new_chords=False, bis=False, chunks=None):
        self.row_type = row_type
        self.new_chords = new_chords
        self.chunks = [] if chunks is None else chunks
        self.bis = bis

    @staticmethod
    def parseDOM(root, bis=False):
        if root.text:
            chunks = [RowChunk(content=root.text)]
        else:
            chunks = []
        for chunk in root.getchildren():
            chunks.append(RowChunk(chord=chunk.attrib['a'], content=chunk.tail))
        return Row(new_chords=bool(strtobool(root.attrib.get('important_over', 'false'))), bis=bis, chunks=chunks)


class BlockType(Enum):
    VERSE = 'V'
    CHORUS = 'C'
    INSTRUMENTAL = 'I'
    OTHER = 'O'

    @staticmethod
    def parse(s):
        return {
            'verse': BlockType.VERSE,
            'chorus': BlockType.CHORUS,
            'instrumental': BlockType.INSTRUMENTAL,
            'other': BlockType.OTHER
        }[s]


class Block:
    def __init__(self, block_type=BlockType.VERSE, rows=None):
        self.block_type = block_type
        self.rows = [] if rows is None else rows

    @staticmethod
    def parseDOM(root, linked=False):
        block_type = BlockType.parse(root.attrib['type'])
        rows = []
        for child in root.getchildren():
            if child.tag == '{http://21wdh.staszic.waw.pl}bis':
                bis_rows = [Row.parseDOM(row, bis=True) for row in child.findall('{*}row')]
                bis_times = int(child.attrib.get('times', '2'))
                bis_rows[-1].bis = bis_times
                rows += bis_rows
            else:
                rows.append(Row.parseDOM(child))
        if len(rows) == 1:
            rows[0].row_type = RowType.SINGLE
        elif len(rows) > 1:
            rows[0].row_type = RowType.FIRST_SPECIAL
            rows[-1].row_type = RowType.LAST
        if linked:
            for row in rows:
                row.new_chords = False
        return Block(block_type=block_type, rows=rows)


class Song:
    def __init__(self, title='', text_author='', composer='', artist='', original_title='', translator='', alias='',
                 comment='', music_source='', album='', blocks=None, metre='', barre=''):
        self.title = title if title else ''
        self.text_author = text_author if text_author else ''
        self.composer = composer if composer else ''
        self.artist = artist if artist else ''
        self.original_title = original_title if original_title else ''
        self.translator = translator if translator else ''
        self.alias = alias if alias else ''
        self.comment = comment if comment else ''
        self.blocks = [] if blocks is None else blocks
        self.music_source = music_source if music_source else ''
        self.album = album if album else ''
        self.metre = metre if metre else ''
        self.barre = barre if barre else ''

    @staticmethod
    def parseDOM(root):
        # A child of 'lyric' element may either be a text block, a reference to a text block (e.g. to a chorus), or a tablature.
        text_blocks = root.findall('{*}lyric/{*}block')

        def elementTextOrNone(elem):
            return elem.text if elem is not None else None

        def get_attrib(elem, a):
            return elem.attrib.get(a) if elem is not None else None

        def flatten(block):
            return Block.parseDOM(block) if 'blocknb' not in block.attrib else Block.parseDOM(
                text_blocks[int(block.attrib['blocknb']) - 1], linked=True)

        blocks = [flatten(block) for block in root.find('{*}lyric').getchildren() if
                  block.tag != '{http://21wdh.staszic.waw.pl}tabbs']

        x = root.xpath("./s:music/s:guitar/@barre", namespaces={"s": "http://21wdh.staszic.waw.pl"})
        return Song(
            title=root.get('title'),
            text_author=elementTextOrNone(root.find('{*}text_author')),
            composer=elementTextOrNone(root.find('{*}composer')),
            artist=elementTextOrNone(root.find('{*}artist')),
            original_title=elementTextOrNone(root.find('{*}original_title')),
            translator=elementTextOrNone(root.find('{*}translator')),
            comment=elementTextOrNone(root.find('{*}comment')),
            alias=elementTextOrNone(root.find('{*}alias')),
            blocks=blocks,
            metre=get_attrib(root.find('{*}music'), 'metre'),
            barre=x[0] if x else None,
            album=elementTextOrNone(root.find('{*}album')),
            music_source=elementTextOrNone(root.find('{*}music_source'))
        )


def parse_song_xml(path):
    tree = etree.parse(path)
    song = Song.parseDOM(tree.getroot())
    return song


In [14]:
df_songs=pd.DataFrame(columns=["title", "lyrics", "text_author", "composer", "artist"])
df_songs

,title,lyrics,text_author,composer,artist


In [15]:
# creating Data Frame df_songs with title amd lyrics, each song in 
for song in songs:
    
    song=parse_song_xml('Data/songs/'+song)
    title=song.title

    lyrics=""
    for block in song.blocks:
        for row in block.rows:
            for chunk in row.chunks:
                lyrics+=chunk.content
            lyrics+=' '
    text_author=song.text_author
    composer=song.composer
    artist=song.artist
    
    new_row={"title": [title], "lyrics": [lyrics], "text_author": [text_author], "composer": [composer], "artist": [artist]}
    
    df_songs=pd.concat([df_songs,pd.DataFrame(new_row)], ignore_index=True)
    

In [16]:
 df_songs

,title,lyrics,text_author,composer,artist
0,Sześć błota stóp,"Sześć błota stóp Sześć błota stóp Uderz w bęben już Bo nadszedł mój czas Wrzućcie mnie do wody Na wieczną wachcę trza nam gdzie Sześć błota stóp Sześć błota stóp Dziewięć sążni wody I sześć błota stóp Ściągnijcie flagę w dół Uszyjcie worek mi Dwie kule przy nogach Ostatni ścieg bez krwi no i Sześć błota stóp Sześć błota stóp Dziewięć sążni wody I sześć błota stóp Na wachcie więcej już Nie ujrzycie chyba mnie Kończę ziemską podróż Do Hilo dotrę wnet tam gdzie Sześć błota stóp Sześć błota stóp Dziewięć sążni wody I sześć błota stóp A ląd daleko jest Przed wami setki mil A mnie pozostało Do lądu kilka chwil no i Sześć błota stóp Sześć błota stóp Dziewięć sążni wody I sześć błota stóp Sześć błota stóp, sześć błota stóp, sześć błota stóp",Henryk Czekała,Cyril Tawney,Mechanicy Shanty
1,Balonik,"Płacze dziewczynka, Balon uciekł jej, Ludzie mówią -- nie płacz, A balonik hen. Płacze dziewczyna, Chłopca trzeba jej, Ludzie mówią -- nie płacz, A balonik hen. Płacze kobieta, Mąż porzucił ją, Ludzie mówią -- nie płacz, A balonik hen. Płacze staruszka, Życie moje wróć! A balonik wrócił I błękitny jest.",Bułat Okudżawa,Bułat Okudżawa,Bułat Okudżawa
2,Nie patrz na to miła,"Świat jest taki a nie inny i zapewne się nie zmieni ogłosili wczoraj słynni dwaj uczeni a w przypadku słynnych ludzi każda myśl jest całkiem dobra więc ci goście otrzymali zaraz Nobla Mieli rację jest jak było czyli całkiem nieciekawie w rowie leży pierwsza miłość po zabawie jak się bawić to się bawić krzyż na szyi w dłoni ""seta"" i na niebie czasem zjawi się kometa Zamknij oczy nie patrz na to moja miła pod powieką znaczenie lepsze są widoki i w tym nasza siła i w tym nasza siła że umiemy jeszcze czasem marzyć Jak się bawić to się bawić a tu biedy coraz więcej i ważniejszy jest karabin niźli serce tutaj się zabawą żyje za to tępe rządzą głowy a czasami kretyn z kijem bejsbolowym Nie patrz miła na to wszystko człowiek musi się szanować niech nie schyla się za nisko twoja głowa lecz na chwilę moment jeden za to z góry ci dziękuję otwórz oczy tylko kiedy mnie całujesz A na resztę zamknij oczy moja miła pod powieką znaczenie lepsze są widoki i w tym nasza siła i w tym nasza siła że umiemy jeszcze czasem marzyć A na resztę zamknij oczy moja miła pod powieką znaczenie lepsze są obrazy i w tym nasza siła i w tym nasza siła że umiemy jeszcze czasem marzyć",Andrzej Sikorowski,Andrzej Sikorowski,Pod Budą
3,Madonna,"Polsko--ruska madonna Polsko--starocerkiewna Z gorejącego złota Z żywego drewna, srebra Bizantyjsko--słowiańska W śnieżnobiałych sukienkach Nie na bielonych płótnach Na trumiennych deskach Polsko--ruska madonna Polsko--starocerkiewna Z gorejącego złota Z żywego drewna, srebra Nie bojarska, nie carska Zasmucona, kamienna twarz Nie bojarska, nie carska Zasmucona, chłopska ikona Polsko--ruska madonna Polsko--starocerkiewna Z gorejącego złota Z żywego drewna, srebra",,,
4,Nie rozdziobią nas kruki,"Nie rozdziobią nas kruki Ni wrony, ani nic! Nie rozszarpią na sztuki Poezji wściekłe kły! Ruszaj się Bruno, idziemy na piwo; Niechybnie brakuje tam nas! Od stania w miejscu niejeden już zginął, Niejeden zginął już kwiat! Nie omami nas forsa Ni sławy pusty dźwięk! Inną ścigamy postać: Realnej zjawy tren! Ruszaj się Bruno, idziemy na piwo; Niechybnie brakuje tam nas! Od stania w miejscu niejeden już zginął, Niejeden zginął już kwiat! Nie zdechniemy tak szybko, Jak sobie roi śmierć! Ziemia dla nas za płytka, Fruniemy w góry gdzieś! Ruszaj się Bruno, idziemy na piwo; Niechybnie brakuje tam nas! Od stania w miejscu niejeden już zginął, Niejeden zginął już kwiat!",Edward Stachura,Krzysztof Myszkowski,Stare Dobre Małżeństwo
...,...,...,...,...,...
448,Noc niespodzianek,"Opowieść będzie to prawdziwa chociaż dziwna Legliśmy w sianie obok legła płeć przeciwna A wiatr złowrogi nucił pieśni swe złowieszcze Że tu się mogą zdarzyć dziwne rzeczy jeszcze To będzie noc pełna niespodzianek W stodole tej na słomie i na sianie Lecz chłopcom 

In [17]:
# text cleaning function
def clean_text(text):
    res = re.sub(r'[^\w\s]', '', text) # removes what is not letter, digit or space
    res = re.sub(r'\d', '', res) # removes digits
    res = re.sub(r'\n', '', res) # removes newlines
    res = re.sub(r' +', ' ', res) # removes unnecessary spaces
    res = res.lower() # changes letters to lowercase
    return res

In [24]:
df_songs1= df_songs
df_songs1.shape

(453, 5)

In [25]:
#Executing the clean_text function on song lyrics
for i in range(0,df_songs1.shape[0]):
    df_songs1.lyrics[i]=clean_text(df_songs1.lyrics[i])

In [26]:
df_songs1

,title,lyrics,text_author,composer,artist
0,Sześć błota stóp,sześć błota stóp sześć błota stóp uderz w bęben już bo nadszedł mój czas wrzućcie mnie do wody na wieczną wachcę trza nam gdzie sześć błota stóp sześć błota stóp dziewięć sążni wody i sześć błota stóp ściągnijcie flagę w dół uszyjcie worek mi dwie kule przy nogach ostatni ścieg bez krwi no i sześć błota stóp sześć błota stóp dziewięć sążni wody i sześć błota stóp na wachcie więcej już nie ujrzycie chyba mnie kończę ziemską podróż do hilo dotrę wnet tam gdzie sześć błota stóp sześć błota stóp dziewięć sążni wody i sześć błota stóp a ląd daleko jest przed wami setki mil a mnie pozostało do lądu kilka chwil no i sześć błota stóp sześć błota stóp dziewięć sążni wody i sześć błota stóp sześć błota stóp sześć błota stóp sześć błota stóp,Henryk Czekała,Cyril Tawney,Mechanicy Shanty
1,Balonik,płacze dziewczynka balon uciekł jej ludzie mówią nie płacz a balonik hen płacze dziewczyna chłopca trzeba jej ludzie mówią nie płacz a balonik hen płacze kobieta mąż porzucił ją ludzie mówią nie płacz a balonik hen płacze staruszka życie moje wróć a balonik wrócił i błękitny jest,Bułat Okudżawa,Bułat Okudżawa,Bułat Okudżawa
2,Nie patrz na to miła,świat jest taki a nie inny i zapewne się nie zmieni ogłosili wczoraj słynni dwaj uczeni a w przypadku słynnych ludzi każda myśl jest całkiem dobra więc ci goście otrzymali zaraz nobla mieli rację jest jak było czyli całkiem nieciekawie w rowie leży pierwsza miłość po zabawie jak się bawić to się bawić krzyż na szyi w dłoni seta i na niebie czasem zjawi się kometa zamknij oczy nie patrz na to moja miła pod powieką znaczenie lepsze są widoki i w tym nasza siła i w tym nasza siła że umiemy jeszcze czasem marzyć jak się bawić to się bawić a tu biedy coraz więcej i ważniejszy jest karabin niźli serce tutaj się zabawą żyje za to tępe rządzą głowy a czasami kretyn z kijem bejsbolowym nie patrz miła na to wszystko człowiek musi się szanować niech nie schyla się za nisko twoja głowa lecz na chwilę moment jeden za to z góry ci dziękuję otwórz oczy tylko kiedy mnie całujesz a na resztę zamknij oczy moja miła pod powieką znaczenie lepsze są widoki i w tym nasza siła i w tym nasza siła że umiemy jeszcze czasem marzyć a na resztę zamknij oczy moja miła pod powieką znaczenie lepsze są obrazy i w tym nasza siła i w tym nasza siła że umiemy jeszcze czasem marzyć,Andrzej Sikorowski,Andrzej Sikorowski,Pod Budą
3,Madonna,polskoruska madonna polskostarocerkiewna z gorejącego złota z żywego drewna srebra bizantyjskosłowiańska w śnieżnobiałych sukienkach nie na bielonych płótnach na trumiennych deskach polskoruska madonna polskostarocerkiewna z gorejącego złota z żywego drewna srebra nie bojarska nie carska zasmucona kamienna twarz nie bojarska nie carska zasmucona chłopska ikona polskoruska madonna polskostarocerkiewna z gorejącego złota z żywego drewna srebra,,,
4,Nie rozdziobią nas kruki,nie rozdziobią nas kruki ni wrony ani nic nie rozszarpią na sztuki poezji wściekłe kły ruszaj się bruno idziemy na piwo niechybnie brakuje tam nas od stania w miejscu niejeden już zginął niejeden zginął już kwiat nie omami nas forsa ni sławy pusty dźwięk inną ścigamy postać realnej zjawy tren ruszaj się bruno idziemy na piwo niechybnie brakuje tam nas od stania w miejscu niejeden już zginął niejeden zginął już kwiat nie zdechniemy tak szybko jak sobie roi śmierć ziemia dla nas za płytka fruniemy w góry gdzieś ruszaj się bruno idziemy na piwo niechybnie brakuje tam nas od stania w miejscu niejeden już zginął niejeden zginął już kwiat,Edward Stachura,Krzysztof Myszkowski,Stare Dobre Małżeństwo
...,...,...,...,...,...
448,Noc niespodzianek,opowieść będzie to prawdziwa chociaż dziwna legliśmy w sianie obok legła płeć przeciwna a wiatr złowrogi nucił pieśni swe złowieszcze że tu się mogą zdarzyć dziwne rzeczy jeszcze to będzie noc pełna niespodzianek w stodole tej na słomie i na sianie lecz chłopcom utrudzonym dziewcząt holowaniem nie w głowie były figle tylko spanie a że dzieczęta to 

In [21]:
df_songs1.lyrics=df_songs1.lyrics.apply(clean_text)

In [22]:
df_songs1

,title,lyrics,text_author,composer,artist
0,Sześć błota stóp,sześć błota stóp sześć błota stóp uderz w bęben już bo nadszedł mój czas wrzućcie mnie do wody na wieczną wachcę trza nam gdzie sześć błota stóp sześć błota stóp dziewięć sążni wody i sześć błota stóp ściągnijcie flagę w dół uszyjcie worek mi dwie kule przy nogach ostatni ścieg bez krwi no i sześć błota stóp sześć błota stóp dziewięć sążni wody i sześć błota stóp na wachcie więcej już nie ujrzycie chyba mnie kończę ziemską podróż do hilo dotrę wnet tam gdzie sześć błota stóp sześć błota stóp dziewięć sążni wody i sześć błota stóp a ląd daleko jest przed wami setki mil a mnie pozostało do lądu kilka chwil no i sześć błota stóp sześć błota stóp dziewięć sążni wody i sześć błota stóp sześć błota stóp sześć błota stóp sześć błota stóp,Henryk Czekała,Cyril Tawney,Mechanicy Shanty
1,Balonik,płacze dziewczynka balon uciekł jej ludzie mówią nie płacz a balonik hen płacze dziewczyna chłopca trzeba jej ludzie mówią nie płacz a balonik hen płacze kobieta mąż porzucił ją ludzie mówią nie płacz a balonik hen płacze staruszka życie moje wróć a balonik wrócił i błękitny jest,Bułat Okudżawa,Bułat Okudżawa,Bułat Okudżawa
2,Nie patrz na to miła,świat jest taki a nie inny i zapewne się nie zmieni ogłosili wczoraj słynni dwaj uczeni a w przypadku słynnych ludzi każda myśl jest całkiem dobra więc ci goście otrzymali zaraz nobla mieli rację jest jak było czyli całkiem nieciekawie w rowie leży pierwsza miłość po zabawie jak się bawić to się bawić krzyż na szyi w dłoni seta i na niebie czasem zjawi się kometa zamknij oczy nie patrz na to moja miła pod powieką znaczenie lepsze są widoki i w tym nasza siła i w tym nasza siła że umiemy jeszcze czasem marzyć jak się bawić to się bawić a tu biedy coraz więcej i ważniejszy jest karabin niźli serce tutaj się zabawą żyje za to tępe rządzą głowy a czasami kretyn z kijem bejsbolowym nie patrz miła na to wszystko człowiek musi się szanować niech nie schyla się za nisko twoja głowa lecz na chwilę moment jeden za to z góry ci dziękuję otwórz oczy tylko kiedy mnie całujesz a na resztę zamknij oczy moja miła pod powieką znaczenie lepsze są widoki i w tym nasza siła i w tym nasza siła że umiemy jeszcze czasem marzyć a na resztę zamknij oczy moja miła pod powieką znaczenie lepsze są obrazy i w tym nasza siła i w tym nasza siła że umiemy jeszcze czasem marzyć,Andrzej Sikorowski,Andrzej Sikorowski,Pod Budą
3,Madonna,polskoruska madonna polskostarocerkiewna z gorejącego złota z żywego drewna srebra bizantyjskosłowiańska w śnieżnobiałych sukienkach nie na bielonych płótnach na trumiennych deskach polskoruska madonna polskostarocerkiewna z gorejącego złota z żywego drewna srebra nie bojarska nie carska zasmucona kamienna twarz nie bojarska nie carska zasmucona chłopska ikona polskoruska madonna polskostarocerkiewna z gorejącego złota z żywego drewna srebra,,,
4,Nie rozdziobią nas kruki,nie rozdziobią nas kruki ni wrony ani nic nie rozszarpią na sztuki poezji wściekłe kły ruszaj się bruno idziemy na piwo niechybnie brakuje tam nas od stania w miejscu niejeden już zginął niejeden zginął już kwiat nie omami nas forsa ni sławy pusty dźwięk inną ścigamy postać realnej zjawy tren ruszaj się bruno idziemy na piwo niechybnie brakuje tam nas od stania w miejscu niejeden już zginął niejeden zginął już kwiat nie zdechniemy tak szybko jak sobie roi śmierć ziemia dla nas za płytka fruniemy w góry gdzieś ruszaj się bruno idziemy na piwo niechybnie brakuje tam nas od stania w miejscu niejeden już zginął niejeden zginął już kwiat,Edward Stachura,Krzysztof Myszkowski,Stare Dobre Małżeństwo
...,...,...,...,...,...
448,Noc niespodzianek,opowieść będzie to prawdziwa chociaż dziwna legliśmy w sianie obok legła płeć przeciwna a wiatr złowrogi nucił pieśni swe złowieszcze że tu się mogą zdarzyć dziwne rzeczy jeszcze to będzie noc pełna niespodzianek w stodole tej na słomie i na sianie lecz chłopcom utrudzonym dziewcząt holowaniem nie w głowie były figle tylko spanie a że dzieczęta to 

In [15]:
df_songs1.to_csv("Data/df_songs1.csv")

In [16]:
DF=pd.read_csv("Data/df_songs1.csv", index_col="Unnamed: 0")

In [17]:
DF

,title,lyrics,text_author,composer,artist
0,Sześć błota stóp,sześć błota stóp sześć błota stóp uderz w bęben już bo nadszedł mój czas wrzućcie mnie do wody na wieczną wachcę trza nam gdzie sześć błota stóp sześć błota stóp dziewięć sążni wody i sześć błota stóp ściągnijcie flagę w dół uszyjcie worek mi dwie kule przy nogach ostatni ścieg bez krwi no i sześć błota stóp sześć błota stóp dziewięć sążni wody i sześć błota stóp na wachcie więcej już nie ujrzycie chyba mnie kończę ziemską podróż do hilo dotrę wnet tam gdzie sześć błota stóp sześć błota stóp dziewięć sążni wody i sześć błota stóp a ląd daleko jest przed wami setki mil a mnie pozostało do lądu kilka chwil no i sześć błota stóp sześć błota stóp dziewięć sążni wody i sześć błota stóp sześć błota stóp sześć błota stóp sześć błota stóp,Henryk Czekała,Cyril Tawney,Mechanicy Shanty
1,Balonik,płacze dziewczynka balon uciekł jej ludzie mówią nie płacz a balonik hen płacze dziewczyna chłopca trzeba jej ludzie mówią nie płacz a balonik hen płacze kobieta mąż porzucił ją ludzie mówią nie płacz a balonik hen płacze staruszka życie moje wróć a balonik wrócił i błękitny jest,Bułat Okudżawa,Bułat Okudżawa,Bułat Okudżawa
2,Nie patrz na to miła,świat jest taki a nie inny i zapewne się nie zmieni ogłosili wczoraj słynni dwaj uczeni a w przypadku słynnych ludzi każda myśl jest całkiem dobra więc ci goście otrzymali zaraz nobla mieli rację jest jak było czyli całkiem nieciekawie w rowie leży pierwsza miłość po zabawie jak się bawić to się bawić krzyż na szyi w dłoni seta i na niebie czasem zjawi się kometa zamknij oczy nie patrz na to moja miła pod powieką znaczenie lepsze są widoki i w tym nasza siła i w tym nasza siła że umiemy jeszcze czasem marzyć jak się bawić to się bawić a tu biedy coraz więcej i ważniejszy jest karabin niźli serce tutaj się zabawą żyje za to tępe rządzą głowy a czasami kretyn z kijem bejsbolowym nie patrz miła na to wszystko człowiek musi się szanować niech nie schyla się za nisko twoja głowa lecz na chwilę moment jeden za to z góry ci dziękuję otwórz oczy tylko kiedy mnie całujesz a na resztę zamknij oczy moja miła pod powieką znaczenie lepsze są widoki i w tym nasza siła i w tym nasza siła że umiemy jeszcze czasem marzyć a na resztę zamknij oczy moja miła pod powieką znaczenie lepsze są obrazy i w tym nasza siła i w tym nasza siła że umiemy jeszcze czasem marzyć,Andrzej Sikorowski,Andrzej Sikorowski,Pod Budą
3,Madonna,polskoruska madonna polskostarocerkiewna z gorejącego złota z żywego drewna srebra bizantyjskosłowiańska w śnieżnobiałych sukienkach nie na bielonych płótnach na trumiennych deskach polskoruska madonna polskostarocerkiewna z gorejącego złota z żywego drewna srebra nie bojarska nie carska zasmucona kamienna twarz nie bojarska nie carska zasmucona chłopska ikona polskoruska madonna polskostarocerkiewna z gorejącego złota z żywego drewna srebra,NaN,NaN,NaN
4,Nie rozdziobią nas kruki,nie rozdziobią nas kruki ni wrony ani nic nie rozszarpią na sztuki poezji wściekłe kły ruszaj się bruno idziemy na piwo niechybnie brakuje tam nas od stania w miejscu niejeden już zginął niejeden zginął już kwiat nie omami nas forsa ni sławy pusty dźwięk inną ścigamy postać realnej zjawy tren ruszaj się bruno idziemy na piwo niechybnie brakuje tam nas od stania w miejscu niejeden już zginął niejeden zginął już kwiat nie zdechniemy tak szybko jak sobie roi śmierć ziemia dla nas za płytka fruniemy w góry gdzieś ruszaj się bruno idziemy na piwo niechybnie brakuje tam nas od stania w miejscu niejeden już zginął niejeden zginął już kwiat,Edward Stachura,Krzysztof Myszkowski,Stare Dobre Małżeństwo
...,...,...,...,...,...
442,Noc niespodzianek,opowieść będzie to prawdziwa chociaż dziwna legliśmy w sianie obok legła płeć przeciwna a wiatr złowrogi nucił pieśni swe złowieszcze że tu się mogą zdarzyć dziwne rzeczy jeszcze to będzie noc pełna niespodzianek w stodole tej na słomie i na sianie lecz chłopcom utrudzonym dziewcząt holowaniem nie w głowie były figle tylko spanie a że dzie

In [18]:
song_names=list(DF.title)
song_names=[x[:35] for x in song_names]
song_names

['Sześć błota stóp',
 'Balonik',
 'Nie patrz na to miła',
 'Madonna',
 'Nie rozdziobią nas kruki',
 'Przechyły',
 'Znów wędrujemy',
 'Makatka z aniołem',
 'Bieszczadzki trakt',
 'Ach, kiedy znowu ruszą dla mnie dni',
 'Wiatr, wiosenny gitarzysta',
 'Dokąd nas zaprowadzisz Panie',
 'Jedyne co mam',
 'Między nami tyle śniegu',
 'Dylan -- pastisz',
 'Wasza Wysokość Kobieto',
 'Chłopaki nie płaczą',
 'Gonić marzenia',
 'Przyjaciółko',
 'Tawerna pod pijaną zgrają',
 'Pocałuj noc',
 'Zwiewność',
 'Żegluj',
 'Easy Rider',
 'Wielki Wóz',
 'Pan Kmicic',
 'Broadway',
 'Już rozpaliło się ognisko',
 'Ci wszyscy ludzie',
 'Jesienne wspominanki',
 'Jaką cenę...',
 'Lekcja historii klasycznej',
 'Miasto',
 'Płacz moja wodo',
 'Szkic do portretu',
 'Piosenka dla Wojtka Bellona',
 'Wojtka Bellona ostatnia ziemska pod',
 'Zabierz mnie na stację',
 'Piosenka dla juniora i jego gitary',
 'Kochać',
 'Jestem mały miś',
 'Krajka (pełna wersja)',
 'Bellonika z miastem',
 'Mucha w szklance lemoniady',
 'Plaste

In [19]:
#polish stopwords from: https://countwordsfree.com/stopwords/polish

f = open("Data/stop_words_polish.txt", "r")
polish_stopwords = f.readlines()
f.close()
polish_stopwords=[x[:-1] for x in polish_stopwords if x[-1:]=='\n']
polish_stopwords

['ach',
 'aj',
 'albo',
 'bardzo',
 'bez',
 'bo',
 'być',
 'ci',
 'cię',
 'ciebie',
 'co',
 'czy',
 'daleko',
 'dla',
 'dlaczego',
 'dlatego',
 'do',
 'dobrze',
 'dokąd',
 'dość',
 'dużo',
 'dwa',
 'dwaj',
 'dwie',
 'dwoje',
 'dziś',
 'dzisiaj',
 'gdyby',
 'gdzie',
 'go',
 'ich',
 'ile',
 'im',
 'inny',
 'ja',
 'ją',
 'jak',
 'jakby',
 'jaki',
 'je',
 'jeden',
 'jedna',
 'jedno',
 'jego',
 'jej',
 'jemu',
 'jeśli',
 'jest',
 'jestem',
 'jeżeli',
 'już',
 'każdy',
 'kiedy',
 'kierunku',
 'kto',
 'ku',
 'lub',
 'ma',
 'mają',
 'mam',
 'mi',
 'mną',
 'mnie',
 'moi',
 'mój',
 'moja',
 'moje',
 'może',
 'mu',
 'my',
 'na',
 'nam',
 'nami',
 'nas',
 'nasi',
 'nasz',
 'nasza',
 'nasze',
 'natychmiast',
 'nią',
 'nic',
 'nich',
 'nie',
 'niego',
 'niej',
 'niemu',
 'nigdy',
 'nim',
 'nimi',
 'niż',
 'obok',
 'od',
 'około',
 'on',
 'ona',
 'one',
 'oni',
 'ono',
 'owszem',
 'po',
 'pod',
 'ponieważ',
 'przed',
 'przedtem',
 'są',
 'sam',
 'sama',
 'się',
 'skąd',
 'tak',
 'taki',
 'tam',
 'ten

In [20]:
lyrics_list=list(DF["lyrics"])
lyrics_list

['sześć błota stóp sześć błota stóp uderz w bęben już bo nadszedł mój czas wrzućcie mnie do wody na wieczną wachcę trza nam gdzie sześć błota stóp sześć błota stóp dziewięć sążni wody i sześć błota stóp ściągnijcie flagę w dół uszyjcie worek mi dwie kule przy nogach ostatni ścieg bez krwi no i sześć błota stóp sześć błota stóp dziewięć sążni wody i sześć błota stóp na wachcie więcej już nie ujrzycie chyba mnie kończę ziemską podróż do hilo dotrę wnet tam gdzie sześć błota stóp sześć błota stóp dziewięć sążni wody i sześć błota stóp a ląd daleko jest przed wami setki mil a mnie pozostało do lądu kilka chwil no i sześć błota stóp sześć błota stóp dziewięć sążni wody i sześć błota stóp sześć błota stóp sześć błota stóp sześć błota stóp ',
 'płacze dziewczynka balon uciekł jej ludzie mówią nie płacz a balonik hen płacze dziewczyna chłopca trzeba jej ludzie mówią nie płacz a balonik hen płacze kobieta mąż porzucił ją ludzie mówią nie płacz a balonik hen płacze staruszka życie moje wróć a ba

In [21]:
tfidf = TfidfVectorizer(stop_words=polish_stopwords) # usuwamy stopwords #mozna usunąć zbyc częste i zbyt rzadkie słowa max_df=20, min_df=3 
tfidf_matrix=tfidf.fit_transform(lyrics_list)
tfidf_matrix.shape #macierz rzadka 

(447, 15146)

In [22]:
def get_svd_matrix(n_components, normalize=False):
    svd = TruncatedSVD(n_components=n_components)
    lsa_matrix = svd.fit_transform(tfidf_matrix)
    print(f"Explained Variance: {sum(svd.explained_variance_ratio_)}")
    if normalize:
        normalizer = Normalizer()
        norm_lsa_matrix = normalizer.fit_transform(lsa_matrix)
        return norm_lsa_matrix
    else:
        return lsa_matrix

In [33]:
norm_lsa_matrix=get_svd_matrix(n_components=400, normalize=True)

Explained Variance: 0.9400873261070547


In [34]:
norm_lsa_matrix.shape

(447, 400)

In [35]:
def plot_dendrogram(lsa_matrix,song_names):

    fig = ff.create_dendrogram(lsa_matrix,
                               orientation='left',
                               labels=song_names,
                               linkagefun=lambda x: sch.linkage(x, 'complete'),
                               distfun=lambda x: ss.distance.pdist(x, metric='euclidean')
                              )
    fig.update_layout(width=900, height=5000)
    fig.show()
    
plot_dendrogram(norm_lsa_matrix, song_names)

Piosenki powstałe na podstawie innej:

   * Opadły mgły i Piosenka z pociągu pośpiesznego "Bieszczady" z Katowic do Zagórza przez Kraków Płaszów, Tarnów, Stróże
   * jesień idzie i wiesiek idzie

Wyłapałam pliki o tej samej/bardzo podobnej treści
Takie same pliki:
    * Pożegnanie 3 i pieśń pożegnalna
    * ja mam tylko jedeń świat i KIedy w piątek
    * pod śliwą oraz anioł i diabeł
    * Krajka i Krajka pełna wersja
    * piosenka do przyjaciela i piosenka dla przyjaciela
    * Jak dobrze nam x2

In [44]:
song="song.xml"

In [46]:
os.path.split(song)

('', 'song.xml')

In [1]:
?exit